In [8]:
from jax import numpy as jnp
import optax
from fused_ce_loss import fused_ce_loss_fwd

In [2]:
from jax import numpy as jnp
from jax import random as jr

vocab_size = 50257
embed_size = 768
batch_size = 32
seq_len = 128

xs = jr.normal(jr.PRNGKey(0), (batch_size, seq_len, embed_size), dtype=jnp.bfloat16)
ys = jr.randint(jr.PRNGKey(1), (batch_size, seq_len), minval=0, maxval=vocab_size)
vocab = jr.normal(jr.PRNGKey(2), (vocab_size, embed_size), dtype=jnp.bfloat16)

In [9]:
def ref_ce_loss(xs, ys, vocab):
    logits = jnp.einsum('b s e, v e->b s v', xs, vocab)
    return optax.losses.softmax_cross_entropy_with_integer_labels(logits, ys)

In [11]:
assert jnp.allclose(ref_ce_loss(xs, ys, vocab), fused_ce_loss_fwd(xs, ys, vocab))

AssertionError: 